 # Load and Trainig the Test data

In [1]:

import graphlab as gl
test_data = gl.SFrame.read_csv("/Users/swastika.b/Documents/Swastika/SantaClaraEdu/courses/Machine Learning/Project/FinalCode/train.csv",verbose= False)
data = gl.SFrame.read_csv("/Users/swastika.b/Documents/Swastika/SantaClaraEdu/courses/Machine Learning/Project/FinalCode/training_data.csv",verbose= False)

[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1503108678.log


This non-commercial license of GraphLab Create for academic use is assigned to sbhat1@scu.edu and will expire on August 12, 2018.


# Preprocessing Data

In [2]:
def transform_text(text):
    
    import re
    import string
    from nltk.tokenize import word_tokenize
    from nltk.corpus import stopwords
    from nltk.stem.porter import PorterStemmer
    
    regex = re.compile('[%s]' % re.escape(string.punctuation))
    porter = PorterStemmer()
    
    lower_case_text = text.translate(None, string.punctuation).lower()
    words_list = word_tokenize(lower_case_text)
    text2 = []
    for word in words_list:
        text0 = word.decode('ascii', 'ignore')
        text1 = regex.sub(u'', text0)
        if not text1 == u'':
            if not text1 in stopwords.words('english'):
                text2.append(porter.stem(text1))
    return text2
    

data['comment_clean'] = data['Comment'].apply(transform_text)

In [3]:
def create_text(words):
    return ' '.join(words)

data['comment_string'] = data['comment_clean'].apply(create_text)

def remove_punctuation(text):
    import string
    lower_case_text = text.translate(None, string.punctuation).lower()
    return lower_case_text
    
#data['comment_string'] = data['Comment'].apply(remove_punctuation)

# Train the Data

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfTransformer

def split_into_lemmas(comments):
    bigram_vectorizer = CountVectorizer(ngram_range=(1, 1), token_pattern=r'\b\w+\b', min_df=1)
    analyze = bigram_vectorizer.build_analyzer()
    return analyze(comments)

vectorizer = CountVectorizer(analyzer=split_into_lemmas,stop_words='english',strip_accents='ascii').fit(data['comment_string'])
text_transformed = vectorizer.transform(data['comment_string'])
tfidf_transformer = TfidfTransformer().fit(text_transformed)
tfidf_transformed_text = tfidf_transformer.transform(text_transformed)



In [5]:
classifier_nb = MultinomialNB(class_prior=[0.50, 0.50]).fit(tfidf_transformed_text, data['Insult'])
classifier_nb.fit(tfidf_transformed_text,data['Insult'])


MultinomialNB(alpha=1.0, class_prior=[0.5, 0.5], fit_prior=True)

# K- Fold Cross Validation

In [6]:
from sklearn.cross_validation import cross_val_predict
from sklearn import metrics
import numpy as np
y = (data['Insult']).to_numpy()
predicted = cross_val_predict(classifier_nb, tfidf_transformed_text, y, cv=10)
metrics.accuracy_score(y, predicted) 

/Users/swastika.b/Documents/Softwares/anaconda/envs/gl-env/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


0.69753914988814314

# Evaluation on Test data

In [7]:
test_data['comment_clean'] = test_data['Comment'].apply(transform_text)
test_data['comment_string'] = test_data['comment_clean'].apply(create_text)
#test_data['comment_string'] = test_data['Comment'].apply(remove_punctuation)

In [8]:
test_text_transformed = vectorizer.transform(test_data['comment_string'])
tfidf_transformed_test_text = tfidf_transformer.transform(test_text_transformed)
predicted_class = classifier_nb.predict(tfidf_transformed_test_text)

In [9]:
import numpy as np
true_class = np.array(test_data['Insult'])
from sklearn.metrics import accuracy_score
accuracy_score(true_class, predicted_class)

0.63060552318216367

# Test on Sample Data

In [10]:
sample_test_data = test_data[30:35]
sample_test_data.select_columns(['Insult','Comment','comment_string'])

Insult,Comment,comment_string
0,"""russellpiano\xa0\nHmmmm...how can we spin Obama ...",russellpianoxa0 hmmmmhowspin obama honor vietnam ...
0,"""A good article, which,for all the good it will ...",good articl good coulddeliv brick wall ...
0,"""Head's upwarning...another source ...",head warninganoth sourcfund media matter peter ...
0,"""@sdnoriko LOL as ifCapcom listens. Watch ...",sdnoriko lol capcomlisten watch fuck hope ...
1,"""Allinit123,your\xa0hypocrisy\xa0is ...",allinit123yourxa0hypocrisyxa0i ...


In [11]:
sample_test_data['comment_string']

dtype: str
Rows: 5
['russellpianoxa0 hmmmmhow spin obama honor vietnam troop someth negativeansw doesnt matter look follow theyll take anyth obama team xa0hmmm make liber messiah look better light revel leak classifi inform new york timesi know let bogu photo shoot viet nam memori make obama look like care vet first well clear vet immedi area cant trust', 'good articl good could deliv brick wall', 'head warninganoth sourc fund media matter peter lewi progess insur compani know auto insur commerci run everi 5 minut cabl televis featur gal look like blowup doll red lip andxc2xa0a perfectli round mouthif know meannif progressivexc2xa0auto insur know part premium go fund progress liberalxc2xa0organ hell bentxc2xa0on destroyingxc2xa0our countri free speech mission toxc2xa0br downxc2xa0hannitybeckdobbslavinlimbaughboortzetc must let win', 'sdnoriko lol capcom listen watch fuck hope wrong love okami', 'allinit123 yourxa0hypocrisyxa0i sicken ignor loser']

In [12]:
sample_text_transformed = vectorizer.transform(sample_test_data['comment_string'])
tfidf_transformed_sample_text = tfidf_transformer.transform(sample_text_transformed)
predicted_class = classifier_nb.predict(tfidf_transformed_sample_text)
classifier_nb.predict(tfidf_transformed_sample_text)

array([0, 0, 0, 0, 1])